In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

Load and prepare data

In [ ]:
return_sequences = False

# Variables relating to the data you want to load
n = 10000
time_span = 5
c = 100
time_step = 0.1
integration_time_step = 0.01
number_timesteps_predict = 1
std = 0

In [ ]:
# Load the observations
name = f"{n}_{time_span}_{str(time_step).replace('.', '')}_{str(integration_time_step).replace('.', '')}_{number_timesteps_predict}_{std}"
dictionary = np.load(f"data_dictionaries/data_{name}.npy", allow_pickle=True).item()
observations = dictionary["observations"]
predictions = dictionary[c]
print(f'observations array shape = {observations.shape}')
print(f'predictions array shape = {predictions.shape}')

In [ ]:
# Split into training and test data

frac = 0.9
num_samples = observations.shape[0]
cut_off = int(frac*num_samples)

train_x = observations[:cut_off,:-number_timesteps_predict, :]
test_x = observations[cut_off:,:-number_timesteps_predict, :]
train_answer = observations[:cut_off, -1, :]
test_answer = observations[cut_off:, -1, :]
train_prediction = predictions[:cut_off, :]
test_prediction = predictions[cut_off:, :]
print(f"shape of training data = {train_x.shape}")
print(f"shape of training predictions = {train_prediction.shape}")
print(f"shape of training answers = {train_answer.shape}")

Create the architecture

In [ ]:
# Create the architecture
model = keras.Sequential()
model.add(layers.LSTM(64, input_shape=(None, 3), return_sequences=return_sequences))
model.add(layers.Dense(16))
model.add(layers.Dense(3))

model.summary()

model.compile(loss='mse', optimizer='adam')

Train

In [ ]:
history = model.fit(train_x, train_answer, epochs=1, validation_data=(test_x, test_answer), batch_size=32, verbose=2, shuffle=True)

In [ ]:
# Plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
ML_predictions = model.predict(test_x)
print(ML_predictions.shape)

In [ ]:
a = 0
b = 2
for i in range(200,210,1):
    plt.scatter(ML_predictions[i,a], ML_predictions[i,b], label = 'ML_prediction', s=12, marker ='x', color = 'b')
    # plt.plot(test_x[i,:,a], test_x[i,:,b], label = 'data', color = 'g')
    plt.plot(observations[9000+i,-2:,a], observations[9000+i,-2:,b], label = 'data', color = 'g')
    plt.scatter(test_answer[i,a], test_answer[i,b], label = 'answer', s=5, color = 'r')
    plt.scatter(test_prediction[i,a], test_prediction[i,b], label = 'prediction', s=10, color = 'y', marker ='o')
    plt.legend()
    plt.show()

Save!

In [29]:
model.save(f'saved_models/timesteps_{number_timesteps_predict}/ML_only_data_{name}.keras')